In [1]:
import logging
logging.getLogger().setLevel(logging.ERROR)
import os 
import glob
from PIL import Image
from typing import List, Optional, Union
import numpy as np
import torch
from metrics.metrics import Metrics, GeometryMetrics, preprocess_image, preprocess_image_rgba, process_folder_new
import json
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

## Compute Metrics for one folder pair

In [10]:
original_folder = "example_data/GT_Test/0ae696bd837219e784b8b7979807184decd5abdb813f0fd7bbfbf6a82bdcc798"
generated_folder = "example_data/Res_Test/0ae696bd837219e784b8b7979807184decd5abdb813f0fd7bbfbf6a82bdcc798"

### Compute Semantic/Quality Metrics
- For evaluations with very small sample size (n<20), set *compute_distribution_metrics = False* as FID & KID require a minimum of 20 samples to be computed

In [11]:
semantic_metrics = process_folder_new(
    original_folder=original_folder,
    generated_folder=generated_folder,
    preprocess_func=preprocess_image,  
    metric_class=Metrics,
    device="cuda",  
    compute_distribution_metrics=False  
)
print("Semantic Metrics:")
for k, v in semantic_metrics.items():
    print(f"{k}: {v}")


Semantic Metrics:
MSE: 1338.7757568359375
CLIP-S: 0.8881114721298218
Spectral_MSE: 129912832.0
D_lambda: 277856960.0
ERGAS: 1.7582569122314453
PSNR: 23.871440887451172
RASE: 294.625244140625
RMSE_wind: 7.454418182373047
SAM: 0.004716605879366398
MS-SSIM: 0.9451460838317871
SSIM: 0.9532877206802368
UQI: -707163904.0
VIF: 0.0716453567147255
LPIPS: 0.024912722408771515
SCC: -0.0031660806853324175


### Compute Geometric Evaluation Metrics

In [12]:
# For geometry metrics:
geometry_metrics = process_folder_new(
    original_folder=original_folder,
    generated_folder=generated_folder,
    preprocess_func=preprocess_image_rgba,  
    metric_class=GeometryMetrics,
    num_points=100  
)
print("Geometry Metrics:")
for k, v in geometry_metrics.items():
    print(f"{k}: {v}")

Geometry Metrics:
Rel_BB_Aspect_Ratio_Diff: 10.64
Rel_Pixel_Area_Diff: 0.12734227006205978
Outline_Normals_Angle_Diff: 14.794582930119809
Squared_Outline_Normals_Angle_Diff: 12.865086102571496
Summed_Outline_Normals_Angle_Diff: 17.751247221891784
Squared_Summed_Outline_Normals_Angle_Diff: 10.507682960998826
Image_Pairs: 2


## Compute Metrics for multiple combinations

In [ ]:
from metrics.metrics_eval import process_metrics_by_viewpoint, tensor_to_serializable, json_file_to_combined_table

### Compute Metrics for a subfolder

In [ ]:
gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
gen_folder = "example_data/Meshfleet_Eval/Results_000"

metrics_result = process_metrics_by_viewpoint(gt_folder, gen_folder, device="cuda")
json_output = json.dumps(tensor_to_serializable(metrics_result), indent=4)

# Save the results in a JSON file in the root folder.
output_file = os.path.join(os.path.dirname(gt_folder), "metrics_results_000.json")
with open(output_file, 'w') as f:
    f.write(json_output)

print(f"Metrics saved to {output_file}")

Processing viewpoint: 000.png
{'MSE': tensor(1075.3264, device='cuda:0'), 'CLIP-S': tensor(0.8792, device='cuda:0'), 'Spectral_MSE': tensor(1.2166e+08, device='cuda:0'), 'D_lambda': tensor(2.0323e+08, device='cuda:0'), 'ERGAS': tensor(1.6201, device='cuda:0'), 'PSNR': tensor(24.2075, device='cuda:0'), 'RASE': tensor(inf, device='cuda:0'), 'RMSE_wind': tensor(5.9141, device='cuda:0'), 'SAM': tensor(0.0073, device='cuda:0'), 'MS-SSIM': tensor(0.9620, device='cuda:0'), 'SSIM': tensor(0.9639, device='cuda:0'), 'UQI': tensor(-5.7838e+08, device='cuda:0'), 'VIF': tensor(0.0737, device='cuda:0'), 'LPIPS': tensor(0.0161, device='cuda:0'), 'SCC': tensor(-0.0005, device='cuda:0'), 'FID': tensor(0.0222, device='cuda:0'), 'IS': tensor(3.0257, device='cuda:0'), 'KID': tensor(0.0892, device='cuda:0')}
{'Rel_BB_Aspect_Ratio_Diff': 9.087826086956522, 'Rel_Pixel_Area_Diff': 0.14957479350625022, 'Outline_Normals_Angle_Diff': 17.288357190070506, 'Squared_Outline_Normals_Angle_Diff': 13.696884473782239, '

### Print metrics from json into a table

In [5]:
json_filepath = "example_data/examples_08082025/Car_Sample_2/scaled/metrics_results.json"
combined_df = json_file_to_combined_table(json_filepath)

print("Combined Metrics Table:")
print(combined_df.to_string())


Combined Metrics Table:
                                           generated_diagonal  generated_rear  generated_side  generated_front
MSE                                              5.812658e+03    8.112500e+03    6.940686e+03     8.605097e+03
CLIP-S                                           8.416202e-01    8.712273e-01    8.857163e-01     8.188758e-01
Spectral_MSE                                     7.561047e+08    1.050878e+09    1.081315e+09     1.649161e+09
D_lambda                                         2.626836e+07    5.425383e+07    8.047733e+07     4.349237e+07
ERGAS                                            5.519731e+00    6.765961e+00    6.245296e+00     7.427383e+00
PSNR                                             1.684072e+01    1.513758e+01    1.662191e+01     1.505695e+01
RASE                                             6.154390e+03    2.959994e+03    4.383693e+03     1.101232e+04
RMSE_wind                                        4.412012e+01    4.945400e+01    4.93065